# Statistical Analysis of Cardiac DNA Synthesis Experiment

------------------------

Welcome to this tutorial on statistical analysis using R. In this notebook, we'll work through an experiment designed to test whether the neurotransmitter **norepinephrine** controls the formation of cardiac muscle cells in rats. This analysis will guide you through data loading, visualization, outlier detection, and various statistical tests, including t-tests and ANOVA.

## Learning Objectives

After completing this tutorial, you will be able to:

1. Load and inspect a dataset in R.
2. Visualize data using box plots and line graphs.
3. Detect and handle outliers appropriately.
4. Perform t-tests and interpret p-values.
5. Conduct ANOVA (Analysis of Variance) tests.
6. Understand and interpret interaction effects in ANOVA.
7. Apply ANCOVA (Analysis of Covariance) to control for confounding variables.

---

Let's get started!

## 1. Experiment Description

**Background:**

- Cardiac muscle cells are formed during development and then permanently exit the mitotic cycle.
- The experiment aims to test whether **norepinephrine** controls the formation of these cells.
- Rats were studied from birth to 19 days of postnatal (PN) age, covering the progression from mitotic to postmitotic growth.

**Experimental Design:**

- On PN1 (Postnatal Day 1), pups received an injection of either **saline** (control) or **6-hydroxydopamine (6OHDA)**, a neurotoxin that destroys norepinephrine neurons.
- At various ages, mitotic activity in the heart was measured by injecting animals with radiolabeled thymidine.
- After 30 minutes, tissues were analyzed for the amount of radiolabel incorporated into DNA relative to total uptake, providing an index of mitotic activity.
- **Negative Controls:**
  - **Liver DNA Synthesis:** Measured in the same animals to check if the effect is specific to the heart.
  - **Protein Synthesis:** Measured in additional pups to see if the effect is specific to DNA synthesis.
- **Measurements:**
  - **Body Weight**
  - **Heart DNA Synthesis (heart incorp/uptake)**
  - **Liver DNA Synthesis (liver incorp/uptake)**

**Questions to Answer:**

1. Does 6OHDA affect the mitotic activity of developing cardiac muscle cells?
2. Is the effect specific to the heart (i.e., not shared by the liver)?
3. Is the effect specific to DNA synthesis (i.e., not shared by protein synthesis)?
4. Is the effect on cardiac DNA synthesis an artifact of reduced body weight caused by 6OHDA?

---

## 2. Load Packages and Data

First, we'll load the necessary packages and import the dataset.

In [1]:
# Suppress warnings and messages
options(warn=-1)
options(jupyter.plot_mimetypes = 'image/png')


# Install and load required packages
if (!require(tidyverse)) install.packages("tidyverse")
if (!require(readxl)) install.packages("readxl")
if (!require(rstatix)) install.packages("rstatix")
if (!require(patchwork)) install.packages("patchwork")




library(tidyverse)
library(readxl)
library(rstatix)
library(patchwork)
library(IRdisplay)


Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: readxl

Loading required package: rstatix


Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter


Loading required package: patchwork



### 2.1 Load the Dataset

Assuming you have the dataset `set1.xlsx`, make sure it's in your working directory. Let's read the data into R.

In [ ]:
# Read the Excel file
data.df <- read_excel("assets/data/set1.xlsx")

# View first few rows of data
head(data.df)

### 2.2 Inspect the Data

Let's check the structure and summary of the data to understand what we're working with.

In [ ]:
# Check the structure of the data
glimpse(data.df)



## Preparing Data for Analysis: Step-by-Step Data Cleaning

First, let's take a look at what we need to accomplish:
1. Create consistent column names
2. Set appropriate data types
3. Organize our columns logically

### 1. Creating Consistent Column Names
First, let's change the column names to something more consistent and easy to work with. R works best with simple, descriptive names that:
- Use lowercase letters
- Replace spaces with underscores (snake_case)
- Avoid special characters
- Clearly describe the content

```r
clean_data.df <- data.df %>%
 rename(
   animal_id = "Animal ID",
   treatment = "PN1 Rx",
   body_weight_g = "bdy wt (g)",
   heart_ratio = "heart incorp/uptake",
   liver_ratio = "liver incorp/uptake",
   sample_number = "#"
 )
 ```

### 2. Converting Data Types
Next, we need to tell R how to interpret our data columns. This is crucial because:
- Factors tell R that a variable is categorical (like treatment groups)
- Numeric values are needed for calculations
- Ordered factors maintain correct sequence (like age groups)

```r
clean_data.df <- clean_data.df %>%
 mutate(
   age_days = parse_number(age),          # Extract numbers from age
   age_factor = fct_reorder(age, parse_number(age)),  # Order age groups correctly
   treatment = factor(treatment),         # Treatment groups are categories
   synthesis = factor(synthesis),         # Type of measurement is categorical
   cohort = factor(cohort)               # Experimental batches are categories
 )
```

### 3. Organizing Columns
Finally, let's arrange our columns in a logical order to make our data frame easier to work with:

```r
clean_data.df <- clean_data.df %>%
 select(
   animal_id, treatment, cohort,          # Identifying information
   age_days, age_factor, synthesis,       # Experimental variables
   sample_number, body_weight_g,          # Additional measurements
   heart_ratio, liver_ratio              # Our main outcome variables
 )
 ```

### 4. Verify Our Work
Always good practice to check the cleaned dataset:

`clean_data.df`

Remember: Proper data cleaning and type setting is crucial for subsequent statistical analysis. Incorrect data types can lead to errors or misleading results in your analysis.

In [ ]:
# Same code as above but we combined into one step using pipes %>%

clean_data.df <- data.df %>%
  # Clean column names using consistent snake_case
  rename(
    animal_id = "Animal ID",
    treatment = "PN1 Rx",
    body_weight_g = "bdy wt (g)",
    heart_ratio = "heart incorp/uptake",
    liver_ratio = "liver incorp/uptake",
    sample_number = "#"
  ) %>%
  # Clean up data types and age
  mutate(
    age_days = parse_number(age),
    age_factor = fct_reorder(age, parse_number(age)),
    treatment = factor(treatment),
    synthesis = factor(synthesis),
    cohort = factor(cohort)
  ) %>%
  # Reorganize columns logically
  select(
    animal_id, treatment, cohort,
    age_days, age_factor, synthesis,
    sample_number, body_weight_g,
    heart_ratio, liver_ratio
  )

# Check our work
head(clean_data.df)



In [ ]:
glimpse(clean_data.df)

## Using Boxplots to Visualize Data Distribution

Boxplots (also called box-and-whisker plots) provide a concise visualization of data distribution. Here's what they show:

- The **box** spans from the first quartile (Q1, 25th percentile) to the third quartile (Q3, 75th percentile)
- The **line** in the box represents the median
- The **whiskers** extend to show the rest of the distribution
- **Points** beyond the whiskers indicate potential outliers (values > 1.5 × IQR from the box edges)

Let's create a boxplot to visualize heart uptake ratios for DNA synthesis:

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6, repr.plot.res=200)

# Heart DNA
clean_data.df %>%
  filter(synthesis == "dna") %>%
  ggplot(aes(x = age_factor, y = heart_ratio, color = treatment)) +
  geom_boxplot() +
  theme_minimal() +
  labs(title = "Heart DNA Synthesis",
       x = "Age",
       y = "Ratio (incorporation/uptake)",
       color = "Treatment")


Now try creating the same boxplot for liver DNA synthesis.


In [ ]:
# Enter your code here


<details>
<summary><strong>Hint:</strong> Click to expand</summary>

The code will be very similar - you'll just need to:
- Replace heart_ratio with liver_ratio
- Update the plot title to reflect liver synthesis
- Update axis labels as needed

</details>



<details>
<summary><strong>Solution:</strong> Click to expand</summary>  

```r  
options(repr.plot.width = 6, repr.plot.height = 5, repr.plot.res=200)

# Liver DNA
clean_data.df %>%
 filter(synthesis == "dna") %>%
 ggplot(aes(x = age_factor, y = liver_ratio, color = treatment)) +
 geom_boxplot() +
 coord_cartesian(ylim = c(0, NA)) +  # Force 0 at bottom, auto-scale top
 theme_minimal() +
 labs(title = "Liver DNA Synthesis",
      x = "Age",
      y = "Ratio (incorporation/uptake)",
      color = "Treatment")
```

</details>

## Make a composite plot of all the different measurements

heart protein ratio, heart DNA ratio, liver protein ratio, liver DNA ratio, body weight

### Using Patchwork to Create Multi-Panel Figures

The patchwork package provides an elegant way to combine multiple ggplot objects. The syntax is intuitive:
- `+` places plots side by side
- `/` places plots one above the other
- Parentheses `()` group plots together

For our five plots, we want this layout:

```text
+----------+----------+
|    p1    |    p2    |  Heart (DNA & Protein)
+----------+----------+
|    p3    |    p4    |  Liver (DNA & Protein)
+----------+----------+
|         p5          |  Body Weight
+----------+----------+
```

The code `(p1 + p2) / (p3 + p4) / p5` creates this arrangement:
1. `p1 + p2` puts heart plots side by side
2. `p3 + p4` puts liver plots side by side
3. The `/` operators stack these rows
4. Final `/p5` puts body weight plot at bottom

Key patchwork operators:
- `+` : side by side
- `/` : top to bottom
- `|` : add a plot independently
- `()` : group plots together

We use `plot_layout(heights = c(1, 1, 1))` to make all rows equal height.



In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10, repr.plot.res=200)

# Heart DNA
p1 <- clean_data.df %>%
  filter(synthesis == "dna") %>%
  ggplot(aes(x = age_factor, y = heart_ratio, color = treatment)) +
  geom_boxplot() +
  theme_minimal() +
  labs(title = "Heart DNA Synthesis",
       x = "Age",
       y = "Ratio (incorporation/uptake)",
       color = "Treatment")

# Heart Protein
p2 <- clean_data.df %>%
  filter(synthesis == "protein") %>%
  ggplot(aes(x = age_factor, y = heart_ratio, color = treatment)) +
  geom_boxplot() +
  theme_minimal() +
  labs(title = "Heart Protein Synthesis",
       x = "Age",
       y = "Ratio (incorporation/uptake)",
       color = "Treatment")

# Liver DNA
p3 <- clean_data.df %>%
  filter(synthesis == "dna") %>%
  ggplot(aes(x = age_factor, y = liver_ratio, color = treatment)) +
  geom_boxplot() +
  theme_minimal() +
  labs(title = "Liver DNA Synthesis",
       x = "Age",
       y = "Ratio (incorporation/uptake)",
       color = "Treatment")

# Liver Protein
p4 <- clean_data.df %>%
  filter(synthesis == "protein") %>%
  ggplot(aes(x = age_factor, y = liver_ratio, color = treatment)) +
  geom_boxplot() +
  theme_minimal() +
  labs(title = "Liver Protein Synthesis",
       x = "Age",
       y = "Ratio (incorporation/uptake)",
       color = "Treatment")

# Body Weight
p5 <- clean_data.df %>%
  ggplot(aes(x = age_factor, y = body_weight_g, color = treatment)) +
  geom_boxplot() +
  theme_minimal() +
  labs(title = "Body Weight",
       x = "Age",
       y = "Body Weight (g)",
       color = "Treatment")

# Arrange plots using patchwork
(p1 + p2) / (p3 + p4) / p5 +
  plot_layout(heights = c(1, 1, 1))

## Understanding Statistical Outliers in Biological Data

#### The Box Plot Method and Biological Context
While statistical methods can identify potential outliers, biological data requires careful consideration beyond pure mathematics. The common box plot method provides a starting point, but should never be used as the sole criterion for data exclusion.

#### Statistical Detection Basics
The box plot method uses these key components:
- Q1: The 25th percentile
- Q3: The 75th percentile
- IQR: The Interquartile Range (Q3 - Q1)

Potential outlier boundaries are calculated as:
- Lower Boundary = Q1 - (1.5 × IQR)
- Upper Boundary = Q3 + (1.5 × IQR)

#### Critical Considerations for Biological Data
When evaluating potential outliers in biological data, consider:

1. **Biological Plausibility**
  - Is the value physically or biologically possible?
  - Example: Synthesis ratios cannot exceed 1.0
  - Values outside known biological limits warrant investigation

2. **Systematic Patterns**
  - Look for consistent patterns across multiple measurements
  - Example: An animal showing consistently small measurements across all parameters may be a runt, not an outlier
  - Single unusual measurements from otherwise normal samples need special scrutiny

3. **Technical vs. Biological Variation**
  - Technical outliers: Values that are impossible or result from clear measurement errors
  - Biological outliers: Unusual but potentially valid biological variation
  - Technical outliers can be removed with more confidence than biological outliers

#### Best Practices for Outlier Analysis

1. **Document Everything**
  - Record all criteria used for outlier identification
  - Note specific justification for each excluded datapoint
  - Maintain transparency in publications about outlier handling

2. **Conservative Approach**
  - Only remove data points when you have strong biological or technical justification
  - Consider reporting results both with and without outliers
  - Be especially cautious with small sample sizes

3. **Multiple Methods**
  - Use statistical methods (like `identify_outliers` in rstatix) as screening tools only
  - Combine statistical analysis with biological expertise
  - Consider consulting subject matter experts for unusual cases

#### Warning Signs That Merit Investigation
- Values outside physically possible ranges
- Measurements that deviate drastically from established literature
- Technical readings beyond instrument capabilities
- Systematic differences across all measurements from a single subject
- Unexplainable extreme values that don't fit known biological mechanisms

Remember: It's better to include a potential outlier than to incorrectly exclude valid biological variation. Statistical outlier detection should guide investigation, not automatically determine exclusion.

In [ ]:
# First let's look at just heart ratio for DNA synthesis
heart_outliers <- clean_data.df %>%
  group_by(synthesis, age_factor, treatment) %>%
  identify_outliers(heart_ratio)

heart_outliers

Now try to identify outliers in liver_ratio using the same approach we used for heart_ratio.


In [ ]:
# Type code here`



<details> <summary><strong>Hint:</strong> Click to expand</summary>  

You'll need to:
- Keep the same grouping structure
- Replace heart_ratio with liver_ratio
- Save to a new variable name (e.g., liver_outliers)

</details>

<details> <summary><strong>Solution:</strong> Click to expand</summary>   

```r
# Look at liver ratio outliers for DNA synthesis
liver_outliers <- clean_data.df %>%
 group_by(synthesis, age_factor, treatment) %>%
 identify_outliers(liver_ratio)

liver_outliers
```

</details>

# Efficient Outlier Detection Using Long Format Data

When working with multiple physiological measurements per animal (heart_ratio, liver_ratio, body_weight_g), converting data to long format enables more precise outlier handling. The key advantage is the ability to selectively mark individual measurements as outliers while preserving other valid measurements from the same animal.

## Data Structure Comparison

### Wide Format (Original)
```
| animal_id | heart_ratio | liver_ratio | body_weight_g |
|-----------|-------------|-------------|---------------|
| 1         | 0.25        | 0.55        | 8.7           |
| 2         | 0.35        | 0.66        | 9.5           |
```

In wide format, marking outliers typically means flagging entire rows. This would force us to discard all measurements from an animal even if only one measurement is problematic.

### Long Format (After pivot_longer)
```
| animal_id | measure       | value  |
|-----------|---------------|--------|
| 1         | heart_ratio   | 0.25   |
| 1         | liver_ratio   | 0.55   |
| 1         | body_weight_g | 8.7    |
| 2         | heart_ratio   | 0.35   |
| 2         | liver_ratio   | 0.66   |
| 2         | body_weight_g | 9.5    |
```

In long format, we can mark individual measurements as outliers. When we pivot back to wide format, these marked outliers become NAs in their respective columns, while preserving other valid measurements from the same animal.

## Benefits of Long Format for Outlier Detection

### Precision in Data Retention
The long format approach allows us to be surgical in our outlier handling. For example, if animal_id 1 has an unusual heart_ratio but normal liver_ratio and body_weight, we can:
1. Mark only the heart_ratio measurement as an outlier
2. Preserve the valid liver_ratio and body_weight measurements
3. When pivoting back to wide format, only the heart_ratio becomes NA

### Statistical Robustness
This approach improves our statistical power by:
- Retaining more valid measurements in the final dataset
- Avoiding unnecessary data loss from otherwise good observations
- Maintaining larger sample sizes for analyses where measurements are valid

### Implementation Efficiency
Processing in long format also streamlines our workflow:
- Apply consistent outlier detection criteria across all measurements
- Group by relevant experimental factors (synthesis, age, treatment) for each measure type
- Maintain data integrity while handling anomalies precisely

In [ ]:
#Convert data to long format
long_data.df <- clean_data.df %>%
  pivot_longer(
    cols = c(heart_ratio, liver_ratio, body_weight_g),
    names_to = "measure",
    values_to = "value"
  ) %>%
  mutate(measure = factor(measure,
                         levels = c("heart_ratio", "liver_ratio", "body_weight_g"),
                         )
        )

head(long_data.df)

In [ ]:
# Find outliers
long_data.df<-long_data.df %>%
group_by(synthesis, measure, age_factor, treatment) %>%
  mutate(
    is_outlier = is_outlier(value),
    is_extreme = is_extreme(value)
  ) %>%
  ungroup()

In [ ]:
# Look at just the outliers
long_data.df %>% filter(is_outlier)

# Outlier Analysis Overview

Before removing outliers, we analyze their patterns to distinguish between technical anomalies and potentially meaningful biological variations. This analysis helps identify:

1. Individual animals with multiple outlier measurements, which could indicate:
   - Biological phenomena (e.g., developmentally delayed "runts")
   - Systematic measurement errors affecting multiple organs
   - Sample handling issues during processing

2. Technical outliers that likely represent measurement errors:
   - Organ-to-body ratios greater than 1 (physically impossible)
   - Inconsistencies between adjacent measurements
   - Values far outside the expected physiological range

## Implementation Approach

The code below creates a summary of outliers by:
1. Calculating interquartile ranges (IQR) within each experimental group
2. Showing the extreme outliers relative to these ranges
3. Counting multiple outliers per animal
4. Preserving contextual information (treatment, age, cohort) for interpretation

This summary allows us to:
- Review outliers in their experimental context
- Identify animals with multiple anomalous measurements
- Make informed decisions about outlier handling based on biological plausibility

The resulting `outlier_summary` dataframe provides a comprehensive view of potential outliers while maintaining the relationship between different measurements from the same animal.

In [ ]:
# Create group-based outlier summary
group_outlier_summary <- long_data.df %>%
 # First get the IQR stats for each group
 group_by(treatment, age_factor, measure, synthesis) %>%
 mutate(
   q1 = quantile(value, 0.25),
   q3 = quantile(value, 0.75),
   iqr = q3 - q1
 ) %>%
 ungroup() %>%
 # Then filter for extreme outliers and add the counts
 filter(is_extreme) %>%
 select(animal_id, treatment, age_factor, cohort, measure, synthesis,
        value, q1, q3, iqr) %>%
 arrange(animal_id, measure) %>%
 group_by(animal_id) %>%
 mutate(n_outliers = n()) %>%
 ungroup()


group_outlier_summary



In [ ]:
html_content<-readr::read_file("assets/html/outlier_animal.html")
display_html(html_content)

In [ ]:
html_content<-readr::read_file("assets/html/outlier_technical.html")
display_html(html_content)

In [ ]:
# Mark the outliers
long_data.df <- long_data.df %>%
  mutate(vetted_outlier = case_when(
    # Mark all measurements from animals 81 and 139 as vetted outliers due to being runts
    animal_id %in% c(81, 139) ~ TRUE,
    # Mark specific measurements from other animals
    animal_id == 54 & measure == "liver_ratio" ~ TRUE,  # excedes max ratio of 1
    animal_id == 55 & measure == "liver_ratio" ~ TRUE,  # unusually low and adjacent to high sample
    animal_id == 55 & measure == "heart_ratio" ~ TRUE,
    animal_id == 65 & measure == "liver_ratio" ~ FALSE,
    animal_id == 167 & measure == "liver_ratio" ~ FALSE,
    # Everything else is not a vetted outlier
    TRUE ~ FALSE
  ))

In [ ]:
#Check the outliers
long_data.df %>% filter(vetted_outlier)

### Data Reshaping for Analysis

Let's walk through how we transform our data for subsequent data analysis steps. R makes these complex transformations straightforward while preserving valuable data!

### Starting Point (Long Format with Outliers Marked)
```
| animal_id | measure      | value  | vetted_outlier |
|-----------|--------------|--------|----------------|
| 1         | heart_ratio  | 0.25   | FALSE          |
| 1         | liver_ratio  | 0.55   | FALSE          |
| 1         | body_weight_g| 8.7    | FALSE          |
| 2         | heart_ratio  | 0.35   | TRUE           |
| 2         | liver_ratio  | 0.66   | FALSE          |
| 2         | body_weight_g| 9.5    | FALSE          |
| 3         | heart_ratio  | 0.28   | FALSE          |
| 3         | liver_ratio  | 1.20   | TRUE           |
| 3         | body_weight_g| 8.9    | FALSE          |
```

### Step 1: Pivot Wider (After Removing Vetted Outliers)
Notice how we keep animal 2's liver data even though its heart measurement was an outlier:


```
| animal_id | heart_ratio | liver_ratio | body_weight_g |
|-----------|-------------|-------------|---------------|
| 1         | 0.25        | 0.55        | 8.7           |
| 2         | NA          | 0.66        | 9.5           |
| 3         | 0.28        | NA          | 8.9           |
```

### Step 2: Create Tissue-Specific Format
The final format preserves all valid measurements for analysis:

```
| animal_id | tissue | ratio | body_weight_g |
|-----------|--------|-------|---------------|
| 1         | heart  | 0.25  | 8.7           |
| 1         | liver  | 0.55  | 8.7           |
| 2         | liver  | 0.66  | 9.5           |
| 3         | heart  | 0.28  | 8.9           |
```


### Why This is Powerful:

1. **Preserve Good Data**
  - We only remove problematic measurements, not entire animals
  - Keep all valid measurements from each animal
  - Maximize our usable data points

2. **Flexible Filtering**
  - Handle outliers at individual measurement level
  - Keep control over which specific data points to exclude
  - Maintain integrity of good measurements

3. **Analysis Ready**
  - Final format perfect for statistical analysis
  - Data structured properly for ANOVA
  - Easy to group by relevant factors

4. **Efficient**
  - What might take hours in Excel takes seconds in R
  - Automated process reduces human error
  - Easy to modify and rerun if needed

5. **Reproducible**
  - Code documents exactly what we did
  - Process can be repeated exactly
  - Clear record of all data transformations

6. **Quality Control**
  - Clear tracking of which measurements were removed
  - Transparent outlier handling
  - Maintains data integrity

This approach maximizes our usable data while maintaining data quality!

In [ ]:
# Pivoting data into correct format
data_vetted.df <- long_data.df %>%
  filter(!vetted_outlier) %>%
  pivot_wider(
    id_cols = c(animal_id, treatment, cohort, age_days, age_factor, synthesis, sample_number),
    names_from = measure,
    values_from = value
  ) %>%
   pivot_longer(
    cols = c(heart_ratio, liver_ratio),
    names_to = "tissue",
    values_to = "ratio",
    names_pattern = "(.+)_ratio"
  ) %>%
  mutate(tissue = factor(tissue)
                         )

head(data_vetted.df)



In [ ]:
glimpse(data_vetted.df)


## Does 6OHDA affect the mitotic activity of developing cardiac muscle cells?

We can first visualize the data using ggplot to assess the effect of 6OHDA on DNA synthesis in cardiac muscle cells.



The `stat_summary()` function in ggplot2 is a powerful tool for statistical visualization. Instead of plotting raw data points directly, it calculates and displays summary statistics. This is particularly useful for showing means, standard errors, and confidence intervals.

In our code, we use multiple layers of stat_summary to build a comprehensive visualization:

1. **Lines connecting means**: `stat_summary(fun.data = mean_se, geom = "line")`
  - Connects mean values across age groups
  - Shows trends over time
  - Helpful for visualizing treatment effects

2. **Error bars**: `stat_summary(fun.data = mean_se, geom = "errorbar")`
  - Shows standard error around means
  - Width parameter controls bar size
  - Critical for showing uncertainty

3. **Mean points**: `stat_summary(fun.data = mean_se, geom = "point")`
  - Marks the exact mean values
  - Makes it easy to read specific points
  - Provides visual anchor for error bars

Common use cases for stat_summary:
- Showing means with confidence intervals
- Displaying medians with quartiles
- Plotting geometric means for log-transformed data
- Creating before/after comparisons
- Visualizing repeated measures designs

The raw data points (geom_point with jitter) are added for transparency, showing the underlying distribution of the data.



In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5, repr.plot.res=240)


data_vetted.df %>%
  filter(synthesis == "dna" , tissue == "heart") %>%
  ggplot(aes(x = age_factor, y = ratio, color = treatment, group = treatment)) +
  stat_summary(fun.data = mean_se, geom = "line") +
  stat_summary(fun.data = mean_se, geom = "errorbar", width = 0.2) +
  stat_summary(fun.data = mean_se, geom = "point") +
geom_point(alpha = 0.3, position = position_jitter(width = 0.2)) +
  theme_minimal() +
  labs(title = "Cardiac Synthesis",
       y = "Heart synthesis (mean ± SE)",
       x = "Age",
       color = "Treatment")

In [ ]:
html_content<-readr::read_file("assets/html/activity_quiz.html")
display_html(html_content)

When standard error (SE) bars do not overlap between two groups, it's a good indication that the difference is likely to be statistically significant at p < 0.05. Looking at our cardiac synthesis data, this suggests significant differences at PN3 and PN5 where the SE bars are clearly separated.

While this visual heuristic is useful for quick interpretation, we'll want to confirm with actual t-tests. However, since we're testing multiple time points, we'll need to consider multiple testing correction to avoid inflating our Type I error rate.

Let's generate the same analysis for Liver DNA synthesis

In [ ]:
# Enter your code here


<details>
<summary><strong>Hint:</strong> Click to expand</summary>

The code will be very similar - you'll just need to:
- Replace the tissue value with liver
- Update the plot title to reflect liver synthesis
- Update axis labels as needed

</details>



<details>
<summary><strong>Solution:</strong> Click to expand</summary>  

```r  
options(repr.plot.width = 6, repr.plot.height = 5, repr.plot.res=240)

# Liver DNA
data_vetted.df %>%
  filter(synthesis == "dna" , tissue == "liver") %>%
  ggplot(aes(x = age_factor, y = ratio, color = treatment, group = treatment)) +
  stat_summary(fun.data = mean_se, geom = "line") +
  stat_summary(fun.data = mean_se, geom = "errorbar", width = 0.2) +
  stat_summary(fun.data = mean_se, geom = "point") +
geom_point(alpha = 0.3, position = position_jitter(width = 0.2)) +
  coord_cartesian(ylim = c(0, NA)) +
  theme_minimal() +
  labs(title = "Liver Synthesis",
       y = "Liver synthesis (mean ± SE)",
       x = "Age",
       color = "Treatment")
```

</details>

<iframe
  src="quiz.html"
  width="100%"
  height="400px"
  frameborder="0">
</iframe>

# Introduction to t-tests and Multiple Testing Correction

## t-test Variants
When performing t-tests to compare groups (e.g., treatment vs control), you'll often see references to both Student's and Welch's t-tests. In practice:

- Welch's t-test has become the standard default in modern statistical software
- The term "t-test" usually refers to either variant without distinction
- Both tests typically produce very similar results in most biological datasets

## Multiple Testing Correction

When performing multiple t-tests on the same dataset (e.g., comparing treatment effects across different ages or tissues), we need to account for the increased probability of false positives. This is often called the multiple testing problem:

- Each t-test has a 5% chance of a false positive (at α = 0.05)
- With 20 tests, we expect 1 false positive by chance alone
- The more tests we perform, the more likely we are to find "significant" results by chance

### Bonferroni Correction
The Bonferroni correction is a straightforward method to control for multiple testing:
- Multiply each p-value by the number of tests performed
- Or divide your significance threshold (α) by the number of tests
- Example: With 10 tests, use α = 0.05/10 = 0.005 as the threshold

This correction is conservative but simple and widely accepted. While other methods exist (like Benjamini-Hochberg), Bonferroni is often preferred when:
- You have a relatively small number of planned comparisons
- Strong control of false positives is important
- You need a method that's easy to explain and interpret

## Implementation
In our analysis, we'll use rstatix's `t_test()` function, which:
- Uses Welch's t-test by default
- Allows easy application of Bonferroni correction
- Provides clear output for reporting results

In [ ]:
data_vetted.df %>%
  filter(synthesis == "dna", tissue== "heart") %>%
  group_by(age_factor) %>%
  t_test(ratio ~ treatment)


In [ ]:
data_vetted.df %>%
  filter(synthesis == "dna", tissue == "heart") %>%
  group_by(age_factor) %>%
  t_test(ratio ~ treatment) %>%
  adjust_pvalue(method = "bonferroni")

The t-test has a key limitation here: it can only compare two groups at a time. For the cardiac synthesis data, understanding both age effect and treatment effect would require multiple t-tests:

1. Between treatments (control vs 6OHDA) at each age point:
   - PN3 control vs 6OHDA
   - PN5 control vs 6OHDA
   - And so on... (6 separate tests)

This approach has two main problems:
1. Multiple testing increases the chance of false positives
2. There's no way to assess how age and treatment interact in a single statistical test

ANOVA is more appropriate because it can:
1. Test all groups in a single statistical test
2. Directly examine both main effects (age, treatment) and their interaction
3. Tell us if the effect of treatment depends on age (which appears true in this data)

So while t-tests are useful for simple comparisons, ANOVA better captures the complexity of this experimental design in a single statistical test.

## Two-Way ANOVA: Testing Treatment Effects Across Age

A two-way ANOVA helps us understand how both treatment and age affect heart DNA synthesis, and importantly, whether the treatment effect changes with age. Using the formula `ratio ~ treatment * age_factor`, we test three hypotheses:

1. **Main Effect of Treatment (H₀)**
  - Null: No difference between treatment groups (averaged across ages)
  - Alternative: Treatment groups differ in their average ratio

2. **Main Effect of Age (H₀)**
  - Null: No difference between age groups (averaged across treatments)
  - Alternative: At least two age groups differ in their ratio

3. **Interaction Effect (Treatment × Age) (H₀)**
  - Null: Treatment effect is consistent across all ages
  - Alternative: Treatment effect varies with age

The `*` in our formula tells R to test both main effects and their interaction. Here's our ANOVA:

In [ ]:
# rstatix version
data_vetted.df %>%
  filter(synthesis == "dna", tissue == "heart") %>%
  anova_test(ratio ~ treatment * age_factor)

### Interpreting the Two-Way ANOVA Results

All three effects in our analysis are statistically significant (p < 0.05):

1. **Treatment Effect** (F(1,106) = 18.42, p < 0.001)
  - Strong evidence that treatment affects heart DNA synthesis
  - Effect size (ges = 0.148) suggests moderate impact
  - Explains about 15% of the variance

2. **Age Effect** (F(5,106) = 55.73, p < 0.001)
  - Very strong evidence that synthesis changes with age
  - Large effect size (ges = 0.724)
  - Age explains about 72% of the variance

3. **Treatment × Age Interaction** (F(5,106) = 9.36, p < 0.001)
  - Strong evidence that treatment effect varies with age
  - Moderate effect size (ges = 0.306)
  - The interaction explains about 31% of the variance

Note: 'ges' is generalized eta-squared, a measure of effect size.

## Is the effect specific to the heart (i.e., not shared by the liver)?

### Repeated Measures ANOVA: Testing Tissue-Specific DNA Synthesis Patterns

### Data Preparation
Analysis restricted to DNA synthesis measurements to examine:
- How treatment affects synthesis across tissues
- Whether tissue responses vary with development
- If treatment effects are tissue-specific

### Data Structure
- Within-subject factor: tissue (heart vs. liver paired measurements)
- Between-subject factors:
 1. Treatment (control vs. 6OHDA)
 2. Age (all developmental timepoints)
- Subject identifier: animal_id

**Between-Subject Null Hypotheses:**
1. **Treatment Effect**: H₀: No difference in DNA synthesis between treated and control animals
2. **Age Effect**: H₀: No difference in DNA synthesis across developmental ages
3. **Treatment × Age**: H₀: Treatment effects are consistent across ages

**Within-Subject Null Hypotheses:**
1. **Tissue Effect**: H₀: No difference in DNA synthesis between heart and liver from same animals
2. **Tissue × Treatment**: H₀: Treatment effects are consistent between paired tissues
3. **Tissue × Age**: H₀: Developmental patterns are consistent between paired tissues
4. **Tissue × Treatment × Age**: H₀: Treatment effects on developmental patterns are consistent between paired tissues

**Statistical Design Considerations:**
- Repeated measures design accounts for paired tissue samples
- Type III sums of squares used by default
- Sphericity not a concern (only two tissue levels)
- Complete pairs required for each subject

In [ ]:
# Run repeated measures ANOVA
rm_anova <- data_vetted.df %>%
  filter(synthesis == "dna") %>%
  anova_test(
    dv = ratio,
    wid = animal_id,
    between = c("treatment", "age_factor"),
    within = tissue
  )

rm_anova

In [ ]:
html_content<-readr::read_file("assets/html/anova_quiz.html")
display_html(html_content)

## Is the effect specific to DNA synthesis (ie not shared by protein synthesis)

### Three-Way ANOVA: Testing Effects of Treatment, Age, and Synthesis Type

**Factor-Level Null Hypotheses:**
1. **Treatment Effect**: H₀: No difference in synthesis ratios between treated and control animals
2. **Age Effect**: H₀: No difference in synthesis ratios across PN5, PN15, and PN19
3. **Synthesis Effect**: H₀: No difference between DNA and protein synthesis ratios

**Two-Way Interaction Null Hypotheses:**
1. **Treatment × Age**: H₀: The relationship between age and synthesis ratios is consistent across treatments
2. **Treatment × Synthesis**: H₀: The effect of treatment is consistent between DNA and protein synthesis
3. **Age × Synthesis**: H₀: Age-related changes are consistent between DNA and protein synthesis

**Three-Way Interaction Null Hypothesis:**
1. **Treatment × Age × Synthesis**: H₀: The effect of 6OHDA on synthesis ratios follows the same age-dependent pattern for both DNA and protein synthesis

**Statistical Design Requirements:**
- Complete pairs of measurements needed
- Balance assessed at each age point
- Type III sums of squares appropriate for slight imbalances

### Check for paired samples and balanced design


In [ ]:
# Check balance of experimental design
data_vetted.df %>%
  filter(tissue == "heart") %>%
  group_by(age_factor, synthesis) %>%
  summarise(n = n(), .groups = "drop") %>%
  spread(synthesis, n)

Initial data review shows synthesis measurements are not uniformly paired:

- PN3: DNA only (n=20)
- PN5: Both DNA (n=20) and protein (n=19)
- PN8: DNA only (n=20)
- PN11: DNA only (n=20)
- PN15: Both DNA (n=19) and protein (n=39)
- PN19: Both DNA (n=20) and protein (n=20)

Therefore, analysis needs to be restricted to ages with both measurements (PN5, PN15, PN19) to maintain valid comparisons.

In [ ]:
data_vetted.df %>%
 filter(tissue == "heart",
        age_factor %in% c("PN5", "PN15", "PN19")) %>%  # Only ages with both measurements
 anova_test(ratio ~ treatment * age_factor * synthesis)

In [ ]:
html_content<-readr::read_file("assets/html/anova3factor_quiz.html")
display_html(html_content)

## Four-Way Mixed ANOVA: Testing Multi-Tissue Response to Treatment

The previous ANOVA's required separate tests to assess the tissue and synthesis specific effects which could lead to type I errors due to multiple hypothesis testing.  We can confirm these results in a single 4 way ANOVA test.

### Data Availability Assessment
Balanced design requirements:
- Only ages with both DNA and protein measurements included (PN5, PN15, PN19)
- Each animal (subject) has measurements for both tissues
- Animal_id used as subject identifier for repeated measures

**Between-Subject Null Hypotheses:**
1. **Treatment Effect**: H₀: No difference in synthesis ratios between treated and control animals
2. **Age Effect**: H₀: No difference in synthesis ratios across PN5, PN15, and PN19
3. **Synthesis Effect**: H₀: No difference between DNA and protein synthesis ratios
4. **Treatment × Age**: H₀: Treatment effects are consistent across ages
5. **Treatment × Synthesis**: H₀: Treatment effects are consistent between synthesis types
6. **Age × Synthesis**: H₀: Age effects are consistent between synthesis types
7. **Treatment × Age × Synthesis**: H₀: Two-way interactions are consistent across third factor

**Within-Subject Null Hypotheses:**
1. **Tissue Effect**: H₀: No difference in synthesis ratios between heart and liver
2. **Tissue × Treatment**: H₀: Tissue differences are consistent across treatments
3. **Tissue × Age**: H₀: Tissue differences are consistent across ages
4. **Tissue × Synthesis**: H₀: Tissue differences are consistent between synthesis types
5. **Three-Way Interactions with Tissue**: H₀: Two-way interactions are consistent across tissues
6. **Four-Way Interaction**: H₀: Three-way interactions are consistent across tissues

**Statistical Design Considerations:**
- Mixed design with repeated measures on tissue
- Type III sums of squares appropriate (default for repeated measures)
- Need to verify complete pairs across tissues

In [ ]:
# Four-factor ANOVA (treatment × age × synthesis × tissue)
# with tissue as repeated measure
data_vetted.df %>%
 filter(age_factor %in% c("PN5", "PN15", "PN19")) %>%  # Only ages with both DNA/protein
 anova_test(
   dv = ratio,
   wid = animal_id,                    # Subject identifier for repeated measures
   between = c("treatment", "age_factor", "synthesis"),  # Between-subject factors
   within = "tissue"                   # Within-subject factor (repeated measure)
 )

In [ ]:
html_content<-readr::read_file("assets/html/anova4factor_quiz.html")
display_html(html_content)

## What about body weight?  




In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6, repr.plot.res=200)

# Heart DNA
data_vetted.df %>%
  ggplot(aes(x = age_factor, y = body_weight_g, color = treatment)) +
  geom_boxplot() +
  theme_minimal() +
  labs(title = "Body Weight",
       x = "Age",
       y = "Body weight (g)",
       color = "Treatment")


In [ ]:
# Perform a two factor ANOVA 
data_vetted.df %>%
  anova_test(log(body_weight_g) ~ treatment * age_factor)

## Is body weight a predictor of cardiac or liver sythesis?

Plot body weight as a function of cardiac DNA synthesis


In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6, repr.plot.res=200)

data_vetted.df %>%
 filter(synthesis == "dna", tissue == "heart") %>%
 ggplot(aes(x = body_weight_g, y = ratio, color = treatment)) +
 geom_point(alpha = 0.5) +
 geom_smooth(method = "lm", se = TRUE) +
 theme_minimal() +
 labs(title = "Heart DNA Synthesis vs Body Weight",
      x = "Body Weight (g)",
      y = "DNA Synthesis Ratio",
      color = "Treatment")




Plot body weight as a function of liver DNA synthesis


In [ ]:
# Enter your code here


<details>
<summary><strong>Hint:</strong> Click to expand</summary>

The code will be very similar - you'll just need to:
- Replace tissue type with liver
- Update the plot title to reflect liver synthesis
- Update axis labels as needed

</details>



<details>
<summary><strong>Solution:</strong> Click to expand</summary>  

```r  
options(repr.plot.width = 6, repr.plot.height = 5, repr.plot.res=200)

# Liver DNA
data_vetted.df %>%
 filter(synthesis == "dna", tissue == "liver") %>%
 ggplot(aes(x = body_weight_g, y = ratio, color = treatment)) +
 geom_point(alpha = 0.5) +
 geom_smooth(method = "lm", se = TRUE) +
 theme_minimal() +
 labs(title = "Liver DNA Synthesis vs Body Weight",
      x = "Body Weight (g)",
      y = "DNA Synthesis Ratio",
      color = "Treatment")
```

</details>

In [ ]:
# Liver DNA
data_vetted.df %>%
 filter(synthesis == "dna", tissue == "liver") %>%
 ggplot(aes(x = body_weight_g, y = ratio, color = treatment)) +
 geom_point(alpha = 0.5) +
 geom_smooth(method = "lm", se = TRUE) +
 theme_minimal() +
 labs(title = "Liver DNA Synthesis vs Body Weight",
      x = "Body Weight (g)",
      y = "DNA Synthesis Ratio",
      color = "Treatment")

In [ ]:
#peform correlation tests on all the combinations
data_vetted.df %>%
  filter(synthesis == "dna") %>%
  group_by(tissue, treatment) %>%
  cor_test(body_weight_g, ratio) %>%
  arrange(tissue, treatment)  # Sort results

# ANCOVA Analysis of Cardiac DNA Synthesis: Controlling for Body Weight Effects

## Research Question and Hypothesis
Can the effect of treatment (6OHDA) on cardiac DNA synthesis  be explained simply by changes in body weight. If the treatment's effect is solely due to reduced body weight, we would expect the relationship between body weight and DNA synthesis to be the same for both control and treated animals - just with treated animals being smaller.

## Statistical Approach
To test this, we use Analysis of Covariance (ANCOVA) which allows us to:
1. Control for body weight as a continuous variable (covariate)
2. Test if treatments differ after accounting for body weight
3. Most importantly, test if the relationship between body weight and DNA synthesis differs between treatments

The ANCOVA model tests three key hypotheses:
- Is there a relationship between body weight and DNA synthesis? (covariate effect)
- Do treatments differ in DNA synthesis when controlling for body weight? (treatment effect)
- Does the relationship between body weight and DNA synthesis differ between treatments? (interaction)

# ANCOVA Setup and Variable Specification

## Model Formula
`ratio ~ body_weight_g + treatment + treatment:body_weight_g`

## Variables
- **Dependent Variable (Y)**: ratio (DNA synthesis ratio, continuous)
- **Covariate (X)**: body_weight_g (continuous)
- **Treatment Factor**: treatment (categorical: control vs 6OHDA)
- **Interaction**: treatment:body_weight_g (tests if slopes differ between treatments)

## Analysis Specifications
- **Type III Sums of Squares**: Used because:
 - Handles unbalanced designs
 - Tests each effect after controlling for all others
 - Appropriate when interactions are present
 - Less sensitive to unequal sample sizes

)

In [ ]:
data_vetted.df %>%
  filter(synthesis == "dna", tissue == "heart") %>%
  anova_test(ratio ~ body_weight_g + treatment + treatment:body_weight_g,
             type = 3)

data_vetted.df %>%
  filter(synthesis == "dna", tissue == "liver") %>%
  anova_test(ratio ~ body_weight_g + treatment + treatment:body_weight_g,
             type = 3)




## Results and Interpretation
Our ANCOVA analysis reveals:

1. **Body Weight Effect** (F(1,114) = 13.914, p = 0.0003)
  - Significant relationship between body weight and DNA synthesis
  - Explains 10.9% of variance (ges = 0.109)
  - Confirms body weight is indeed related to DNA synthesis

2. **Treatment Effect** (F(1,114) = 19.099, p = 0.00003)
  - Significant difference between treatments after controlling for body weight
  - Largest effect, explaining 14.3% of variance (ges = 0.143)
  - Shows treatment affects DNA synthesis beyond just weight changes

3. **Body Weight × Treatment Interaction** (F(1,114) = 11.999, p = 0.0008)
  - Significant interaction indicates different slopes between treatments
  - Explains 9.5% of variance (ges = 0.095)
  - **Critical finding**: The relationship between body weight and DNA synthesis differs between treatments

## Conclusion
The significant interaction proves that body weight reduction alone cannot explain the treatment effect on cardiac DNA synthesis. If the treatment effect was simply due to reduced body weight, we would expect parallel regression lines (no interaction). Instead, we find that treatment fundamentally alters how DNA synthesis relates to body weight, indicating a more complex biological effect beyond simple growth reduction.